In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import datetime
import matplotlib.pyplot as plt
import contextily as cx

## 读取数据 #df

In [2]:
path = 'process data/dataset2/'
df = pd.read_csv(path+'dataset_2.csv', usecols=[0,3,4])
df

,device_id,device_home_latitude,device_home_longitude
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036
1,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
3,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
4,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
...,...,...,...
59839133,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321534,114.260255
59839134,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321534,114.260255
59839135,b459657d-9ab1-4f73-9154-28f6044e2df7,22.321534,114.260255
59839136,b4797889-7ba8-6302-2099-7fd7d0cf6e40,22.266985,114.234298


In [3]:
df = df.drop_duplicates()
df

,device_id,device_home_latitude,device_home_longitude
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828
10,f68a046d-48d6-41bc-a8dc-8210b026d2a7,22.407392,114.224594
14,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,22.324517,114.160602
15,e9872505-727b-42f7-b4d5-efe02dc948d7,22.336465,114.245417
...,...,...,...
54706209,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,22.367530,114.127846
55123313,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,22.469127,114.006127
55997466,a4158a52-3c61-4f02-abbb-f035faa6faee,22.257824,114.165734
56447442,eebeebff-812f-4be2-aaa4-94f86c528427,22.290000,114.150000


In [4]:
duplicates = df.duplicated(subset='device_id')
df[duplicates]
## 无重复deviceid

,device_id,device_home_latitude,device_home_longitude


In [5]:
device_cnt = np.size(df['device_id'].unique())
device_cnt
## 有住址经纬度的用户有 105037 个

105037

In [12]:
## 没有重复值，保存数据：
df.to_csv(path + "devics_lat_lon.csv", index=False)

## 将 device经纬度 转换为 GeoDataFrame #device_lat_lon_gdf

In [6]:
geometry = gpd.points_from_xy(df['device_home_longitude'], df['device_home_latitude'])
device_lat_lon_gdf = gpd.GeoDataFrame(df, geometry=geometry)
device_lat_lon_gdf.crs = 'EPSG:4326'
device_lat_lon_gdf

,device_id,device_home_latitude,device_home_longitude,geometry
0,f654d4a6-3850-4021-98df-1e4de5d5a945,22.378021,114.258036,POINT (114.25804 22.37802)
2,f6580b80-1451-48b9-b99f-685c17a54fc8,22.365877,114.136828,POINT (114.13683 22.36588)
10,f68a046d-48d6-41bc-a8dc-8210b026d2a7,22.407392,114.224594,POINT (114.22459 22.40739)
14,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,22.324517,114.160602,POINT (114.16060 22.32452)
15,e9872505-727b-42f7-b4d5-efe02dc948d7,22.336465,114.245417,POINT (114.24542 22.33646)
...,...,...,...,...
54706209,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,22.367530,114.127846,POINT (114.12785 22.36753)
55123313,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,22.469127,114.006127,POINT (114.00613 22.46913)
55997466,a4158a52-3c61-4f02-abbb-f035faa6faee,22.257824,114.165734,POINT (114.16573 22.25782)
56447442,eebeebff-812f-4be2-aaa4-94f86c528427,22.290000,114.150000,POINT (114.15000 22.29000)


In [7]:
device_lat_lon_gdf = device_lat_lon_gdf.drop(['device_home_latitude','device_home_longitude'], axis=1)
device_lat_lon_gdf

,device_id,geometry
0,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
2,f6580b80-1451-48b9-b99f-685c17a54fc8,POINT (114.13683 22.36588)
10,f68a046d-48d6-41bc-a8dc-8210b026d2a7,POINT (114.22459 22.40739)
14,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,POINT (114.16060 22.32452)
15,e9872505-727b-42f7-b4d5-efe02dc948d7,POINT (114.24542 22.33646)
...,...,...
54706209,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,POINT (114.12785 22.36753)
55123313,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,POINT (114.00613 22.46913)
55997466,a4158a52-3c61-4f02-abbb-f035faa6faee,POINT (114.16573 22.25782)
56447442,eebeebff-812f-4be2-aaa4-94f86c528427,POINT (114.15000 22.29000)


## 读取building shape file #building_shp

In [8]:
## 读取香港building shape file
shapefile = gpd.read_file('Building_SHP/BUILDING_STRUCTURE.shp', encoding="GBK")
shapefile = shapefile.to_crs('EPSG:4326')
building_shp = shapefile[['OBJECTID', 'geometry']]
building_shp

,OBJECTID,geometry
0,1,"POLYGON ((113.84091 22.21870, 113.84090 22.218..."
1,2,"POLYGON ((113.84094 22.21863, 113.84095 22.218..."
2,3,"POLYGON ((113.84090 22.21867, 113.84091 22.218..."
3,4,"POLYGON ((113.84097 22.21871, 113.84101 22.218..."
4,5,"POLYGON ((113.84102 22.21870, 113.84101 22.218..."
...,...,...
340747,340748,"POLYGON ((114.43641 22.53999, 114.43638 22.539..."
340748,340749,"POLYGON ((114.43640 22.54116, 114.43640 22.541..."
340749,340750,"POLYGON ((114.43668 22.54119, 114.43659 22.541..."
340750,340751,"POLYGON ((114.43675 22.54050, 114.43670 22.540..."


## 关联building_shp和device_lat_lon_gdf #merged

In [9]:
merged = gpd.sjoin(building_shp, device_lat_lon_gdf, how='inner', predicate='intersects')
merged = merged.drop('index_right', axis=1)
merged

,OBJECTID,geometry,device_id
310,311,"POLYGON ((113.85992 22.25518, 113.85987 22.255...",9d84fd00-873a-4473-9c17-42bbf0402455
483,484,"POLYGON ((113.86199 22.25448, 113.86187 22.254...",7da1b9a7-3908-475e-a766-638b5d760eb7
573,574,"POLYGON ((113.86239 22.25465, 113.86238 22.254...",4b625d9c-fc79-465f-8315-8f39cca15e0e
698,699,"POLYGON ((113.86287 22.24768, 113.86284 22.247...",8f47afdb-70aa-438b-a65b-c23e70f1a7be
805,806,"POLYGON ((113.86318 22.25502, 113.86318 22.255...",eb9023f1-b06a-4e4f-8a53-cae203f32b24
...,...,...,...
338948,338949,"POLYGON ((114.31958 22.44845, 114.31961 22.448...",0efd6c0d-6f95-47b9-a4b6-f0724a970a8f
339357,339358,"POLYGON ((114.32680 22.43128, 114.32686 22.431...",908023fa-e3a1-4a58-8b4c-c8b358ce252a
339447,339448,"POLYGON ((114.32859 22.45994, 114.32859 22.460...",9c63161a-3895-4dc9-820d-2de8fbeae333
339848,339849,"POLYGON ((114.35282 22.34764, 114.35281 22.347...",b2a074a2-fd10-463b-adc3-14288ae32f9c


In [23]:
gpd.sjoin(building_shp, device_lat_lon_gdf, how='right', predicate='intersects')

## 2/3 用户无法映射到 building

,index_left,OBJECTID,device_id,geometry
0,NaN,NaN,f654d4a6-3850-4021-98df-1e4de5d5a945,POINT (114.25804 22.37802)
2,NaN,NaN,f6580b80-1451-48b9-b99f-685c17a54fc8,POINT (114.13683 22.36588)
10,301932.0,301933.0,f68a046d-48d6-41bc-a8dc-8210b026d2a7,POINT (114.22459 22.40739)
14,225980.0,225981.0,f6a013e3-3b28-6d4d-bf34-ab0f00d3493a,POINT (114.16060 22.32452)
15,NaN,NaN,e9872505-727b-42f7-b4d5-efe02dc948d7,POINT (114.24542 22.33646)
...,...,...,...,...
54706209,179914.0,179915.0,a88227b1-8718-49a2-a5d0-5e1e5dcfa68e,POINT (114.12785 22.36753)
55123313,NaN,NaN,8db9bafa-5222-4f46-b5b2-49a3d6519dfc,POINT (114.00613 22.46913)
55997466,NaN,NaN,a4158a52-3c61-4f02-abbb-f035faa6faee,POINT (114.16573 22.25782)
56447442,NaN,NaN,eebeebff-812f-4be2-aaa4-94f86c528427,POINT (114.15000 22.29000)


In [10]:
device_building_home_df = merged[['device_id','OBJECTID']]
device_building_home_df

## 105037 个有住址的用户中， 可以获得building 的有 36097 个

,device_id,OBJECTID
310,9d84fd00-873a-4473-9c17-42bbf0402455,311
483,7da1b9a7-3908-475e-a766-638b5d760eb7,484
573,4b625d9c-fc79-465f-8315-8f39cca15e0e,574
698,8f47afdb-70aa-438b-a65b-c23e70f1a7be,699
805,eb9023f1-b06a-4e4f-8a53-cae203f32b24,806
...,...,...
338948,0efd6c0d-6f95-47b9-a4b6-f0724a970a8f,338949
339357,908023fa-e3a1-4a58-8b4c-c8b358ce252a,339358
339447,9c63161a-3895-4dc9-820d-2de8fbeae333,339448
339848,b2a074a2-fd10-463b-adc3-14288ae32f9c,339849


In [11]:
## 保存数据：device_building_home
device_building_home_df.to_csv(path + "device_building_home.csv", index=False)